In [1]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [2]:
dfAll = pd.read_pickle("PklData/AllData.pkl")
dfAll.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
0,15,1547131961064,0,34,0,0,0,False,False,"[2, 1, 0, 182, 3, 161, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,15,1547131961100,0,34,0,0,0,False,False,"[2, 1, 0, 182, 3, 161, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,15,1547131961131,0,34,0,0,0,False,False,"[2, 1, 0, 182, 3, 161, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,15,1547131961165,0,34,0,0,0,False,False,"[2, 1, 0, 182, 3, 161, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,15,1547131961215,0,34,0,0,0,False,False,"[2, 1, 0, 182, 3, 161, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [3]:
df_actual = dfAll[(dfAll.Actual_Data == True) & (dfAll.Is_Pause == False)]
df_actual.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
10422,15,1547132389185,1,680,9,2,0,True,False,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ..."
10423,15,1547132389223,1,680,9,2,0,True,False,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ..."
10424,15,1547132389263,1,680,9,2,0,True,False,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ..."
10425,15,1547132389303,1,680,9,2,0,True,False,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ..."
10426,15,1547132389345,1,680,9,2,0,True,False,"[0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, ..."


In [4]:
print("all: %s, actual data: %s" % (len(dfAll), len(df_actual)))

all: 956426, actual data: 806891


In [5]:
"""%%time
# this will be the faster repetition check
def is_max(row):
    return row.RepetitionID == df_filtered[(df_filtered.userID == row.userID) & (df_filtered.TaskID == row.TaskID) & (df_filtered.VersionID == row.VersionID)].RepetitionID.max()

pool = Pool(8)
df_filtered = df_actual.copy(deep=True)

max_vals = pool.map(is_max, [row for index, row in df_filtered.iterrows()])
df_filtered["IsMax"] = max_vals
print(df_filtered.head())
pool.close()"""

'%%time\n# this will be the faster repetition check\ndef is_max(row):\n    return row.RepetitionID == df_filtered[(df_filtered.userID == row.userID) & (df_filtered.TaskID == row.TaskID) & (df_filtered.VersionID == row.VersionID)].RepetitionID.max()\n\npool = Pool(8)\ndf_filtered = df_actual.copy(deep=True)\n\nmax_vals = pool.map(is_max, [row for index, row in df_filtered.iterrows()])\ndf_filtered["IsMax"] = max_vals\nprint(df_filtered.head())\npool.close()'

In [ ]:
%%time
df_filtered = df_actual.copy(deep=True)
for User in df_filtered.userID.unique():
    print(User)
    for Task in df_filtered.TaskID.unique():
        for Version in df_filtered.VersionID.unique():
            max_rep = df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version)].RepetitionID.max()
            df_filtered = df_filtered.drop(df_filtered[(df_filtered.userID == User) & (df_filtered.TaskID == Task) & (df_filtered.VersionID == Version) & (df_filtered.RepetitionID != max_rep)].index)

15
3
7
16


In [ ]:
df_filtered.to_pickle("PklData/dfFiltered_LSTM.pkl")

In [ ]:
print("actual: %s, filtered data: %s" % (len(df_actual), len(df_filtered)))